First, import the file.

In [2]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
tcga = pd.read_csv("TCGA_all_tidy_with_metadata.csv")

Then, remove extra columns.

In [3]:
tcga_new = tcga.iloc[:, 3:]
gender_col = tcga_new.pop('gender')
tcga_new.insert(0, 'gender', gender_col)
tcga_new.head()

,gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,...,Subtype_miRNA,Subtype_mRNA,Subtype_other,Subtype_protein,treatment_outcome_first_course,Subtype_Selected,vital_status,tumor_status,_PATIENT,_primary_disease
0,FEMALE,6.2537,4.7431,0.0,12.3405,10.2326,9.9114,12.6294,4.8144,0.7471,...,miRNA_1,steroid-phenotype-high+proliferation,C1A,NaN,Progressive Disease,ACC.CIMP-intermediate,Dead,WITH TUMOR,TCGA-OR-A5LC,adrenocortical cancer
1,MALE,8.6468,4.4576,0.0,12.1763,10.2696,9.9354,12.6974,4.2702,0.5323,...,miRNA_4,steroid-phenotype-high,C1A,NaN,Progressive Disease,ACC.CIMP-intermediate,Dead,WITH TUMOR,TCGA-OR-A5JJ,adrenocortical cancer
2,MALE,8.0773,5.2181,0.0,13.4269,10.4399,10.4480,12.6752,1.8429,0.0000,...,miRNA_6,steroid-phenotype-low,C1B,2.0,Complete Remission/Response,ACC.CIMP-intermediate,Alive,TUMOR FREE,TCGA-OR-A5K3,adrenocortical cancer
3,MALE,8.5349,2.4357,0.0,12.3032,10.3281,10.4491,12.6055,5.0432,0.0000,...,miRNA_5,steroid-phenotype-low,C1B,2.0,Complete Remission/Response,ACC.CIMP-high,Alive,TUMOR FREE,TCGA-PK-A5HA,adrenocortical cancer
4,FEMALE,8.9196,5.8492,0.0,12.4274,10.4852,11.0181,13.1200,2.4917,0.0000,...,miRNA_6,steroid-phenotype-low,C1B,2.0,Complete Remission/Response,ACC.CIMP-low,Alive,TUMOR FREE,TCGA-OR-A5LN,adrenocortical cancer


Now we will remove ALL columns except for the gene expression values and the sex.

In [4]:
tcga_newone = tcga_new.iloc[:, 0:20533]
tcga_newone.head()

,gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,...,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS,age_at_initial_pathologic_diagnosis,birth_days_to
0,FEMALE,6.2537,4.7431,0.0,12.3405,10.2326,9.9114,12.6294,4.8144,0.7471,...,4.2718,0.0000,9.4082,12.1791,8.1493,12.3512,2.1349,11.1654,71.0,-26084.0
1,MALE,8.6468,4.4576,0.0,12.1763,10.2696,9.9354,12.6974,4.2702,0.5323,...,2.0439,0.5323,9.6530,11.0509,7.5406,12.6932,2.1921,12.0223,65.0,-24082.0
2,MALE,8.0773,5.2181,0.0,13.4269,10.4399,10.4480,12.6752,1.8429,0.0000,...,3.3299,0.0000,8.8941,11.3881,6.4961,13.4630,2.9008,10.1720,53.0,-19375.0
3,MALE,8.5349,2.4357,0.0,12.3032,10.3281,10.4491,12.6055,5.0432,0.0000,...,4.4560,0.0000,8.8769,11.2305,8.4327,12.1661,5.4951,9.4352,63.0,-23345.0
4,FEMALE,8.9196,5.8492,0.0,12.4274,10.4852,11.0181,13.1200,2.4917,0.0000,...,4.5214,0.0000,9.4850,12.4978,8.5260,12.4398,1.7277,10.6716,31.0,-11414.0


In [5]:
tcga_newone.replace({'MALE': 1, 'FEMALE': 0}, inplace = True)

In [6]:
tcga_smaller = tcga_newone.iloc[:, 0:3500]

Next, we will do correlation on the first 3500 columns of this dataset to see what might be good predictors of gender.

In [7]:
tcga_corr = tcga_smaller.corr(numeric_only=True).abs()[["gender"]]

In [8]:
tcga_model = tcga_smaller[['gender', 'EIF1AY', 'XIST']].dropna()

Finally, we will see how good the model is at predicting gender.

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tcga_model[['EIF1AY', 'XIST']], tcga_model.gender, test_size=0.2, random_state=0)


In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [11]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [12]:
from sklearn import metrics
predict_test = model.predict(X_test)
print(metrics.accuracy_score(y_test, predict_test))

0.994869771112865


With expression data from just two genes, EIF1AY and XIST, we can predict if a person is male or female to 99.5% accuracy. 